# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
# import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

# Import API key
from api_keys  import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/georg/git repositories/module 6/output_data/cities_weather.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,ust'-uda,54.1742,103.0304,23.58,30,15,1.12,RU,1718503170
1,atafu village,-8.5421,-172.5159,28.83,73,23,4.52,TK,1718503171
2,aasiaat,68.7098,-52.8699,2.60,88,18,2.77,GL,1718503173
3,grytviken,-54.2811,-36.5092,-4.84,78,20,4.45,GS,1718503174
4,coquimbo,-29.9533,-71.3436,13.85,79,0,2.57,CL,1718503135


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
print(city_data_df)

                   City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
City_ID                                                                      
0              ust'-uda  54.1742  103.0304     23.58        30          15   
1         atafu village  -8.5421 -172.5159     28.83        73          23   
2               aasiaat  68.7098  -52.8699      2.60        88          18   
3             grytviken -54.2811  -36.5092     -4.84        78          20   
4              coquimbo -29.9533  -71.3436     13.85        79           0   
...                 ...      ...       ...       ...       ...         ...   
542              panjab  34.3880   67.0233     11.65        52           0   
543              mudgee -32.5943  149.5871     11.46        52           3   
544            yuanping  38.7153  112.7575     26.27        31          69   
545              namibe -15.1961   12.1522     19.44        77           0   
546      mazar-e sharif  36.7090   67.1109     31.80        16  

In [4]:
fig = px.scatter_mapbox(city_data_df, lat='Lat', lon='Lng', hover_name='City', size='Humidity',
                        color='Humidity', color_continuous_scale=px.colors.cyclical.IceFire, size_max=5, zoom=3)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria (example: Max Temp between 20 and 30 degrees Celsius, and Humidity less than 50)
criteria_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,ust'-uda,54.1742,103.0304,23.58,30,15,1.12,RU,1718503170
33,mokhsogollokh,61.4681,128.9203,21.24,48,100,3.88,RU,1718503219
43,tindouf,27.6711,-8.1474,28.25,23,29,3.60,DZ,1718503235
77,yanykurgan,43.9150,67.2481,30.00,21,2,2.24,KZ,1718503284
91,bilibino,68.0546,166.4372,21.10,27,81,1.79,RU,1718503303


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = criteria_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,ust'-uda,RU,54.1742,103.0304,30,
33,mokhsogollokh,RU,61.4681,128.9203,48,
43,tindouf,DZ,27.6711,-8.1474,23,
77,yanykurgan,KZ,43.9150,67.2481,21,
91,bilibino,RU,68.0546,166.4372,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import matplotlib.pyplot as plt
# Import API key
from api_keys import geoapify_key

# Set parameters to search for a hotel
radius = 10000  # Define the radius in meters for the hotel search
parameters = {
    "categories": "accommodation.hotel",  # Define the type of place you're looking for, in this case, lodging (i.e., hotels)
    "limit": 1,  # Limit the number of results to 1, as you only want to find the nearest hotel
    "apiKey": geoapify_key  # Add your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    parameters["filter"] = f"circle:{longitude},{latitude},{radius}"
    parameters["bias"] = f"proximity:{longitude},{latitude}"
    
    # print(parameters)
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=parameters)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name']]




Starting hotel search
ust'-uda - nearest hotel: No hotel found
mokhsogollokh - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
yanykurgan - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
lido di ostia - nearest hotel: Fly Decò Hotel
ramos arizpe - nearest hotel: Quinta Royal
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
jalai nur - nearest hotel: 满洲里福门客栈
badger - nearest hotel: Clarion Hotel & Suites
coxim - nearest hotel: Avendita
waddan - nearest hotel: No hotel found
bayganin - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
kyren - nearest hotel: No hotel found
ouarzazat - nearest hotel: Fint
xilin hot - nearest hotel: Hotel hailibu 120
tadmur - nearest hotel: فندق فيلا بالميرا
buka - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
ar rutbah - nearest hotel: No hotel found
murzuq - nearest hotel: فندق باريس - مرزق
caldwell - nearest hotel: Best Western Plus Caldwell Inn & Suites
sa

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,ust'-uda,RU,54.1742,103.0304,30,No hotel found
33,mokhsogollokh,RU,61.4681,128.9203,48,No hotel found
43,tindouf,DZ,27.6711,-8.1474,23,محمد بوسبي
77,yanykurgan,KZ,43.9150,67.2481,21,No hotel found
91,bilibino,RU,68.0546,166.4372,27,No hotel found
98,lido di ostia,IT,41.7321,12.2765,3,Fly Decò Hotel
138,ramos arizpe,MX,25.5500,-100.9667,40,Quinta Royal
139,college,US,64.8569,-147.8028,42,BEST WESTERN PLUS Pioneer Park Inn
158,jalai nur,CN,49.4500,117.7000,34,满洲里福门客栈


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Creating a DataFrame with the provided data
data = {
    'City': ['New York', 'Los Angeles', 'Tokyo'],
    'Country': ['USA', 'USA', 'Japan'],
    'Hotel Name': ['Hotel A', 'No hotel found', 'Hotel B'],
    'Lat': [40.7128, 34.0522, 35.6895],
    'Lng': [-74.0060, -118.2437, 139.6917],
    'Humidity': [85, 78, 90]
}

city_data_df = pd.DataFrame(data)

# Creating the scatter map
fig = px.scatter_mapbox(city_data_df, 
                       lat='Lat', 
                       lon='Lng', 
                       hover_name='City', 
                       hover_data=['Country', 'Hotel Name'], 
                       size='Humidity', 
                       color='Humidity', 
                       color_continuous_scale=px.colors.sequential.Bluered, 
                       size_max=15, 
                       zoom=3)

# Updating the map layout to use OpenStreetMap tiles and adjusting margins
fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":0,"l":0,"b":0},
    title="City Humidity Levels"
)

# Displaying the map
fig.show()

